In [1]:
import socket
import requests

def get_public_ip():
    """
    Optionally, you can use this function to get your own public IP address.
    For connecting to a remote server, you need the server's public IP.
    """
    try:
        response = requests.get("https://api.ipify.org?format=json")
        response.raise_for_status()
        print(f'IP address is: {response.json()["ip"]}')
        return response.json()["ip"]
    except Exception as e:
        print("Error fetching public IP:", e)
        return None

def start_client(server_ip, port=12345):
    # Create a TCP socket
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # Connect to the server using its public IP and the designated port
    client_socket.connect((server_ip, port))
    print(f"Connected to server at {server_ip}:{port}")
    
    # Receive message from the server
    data = client_socket.recv(1024)
    print("Received from server:", data.decode())
    
    # Send a reply to the server
    client_socket.sendall(b"Hello from client!")
    
    # Close the connection
    client_socket.close()

if __name__ == '__main__':
    # For the client, you need to input the server's public IP address.
    # server_ip = get_public_ip() # input("Enter the server's public IP address: ").strip()
    # start_client('127.0.0.1')
    get_public_ip()


IP address is: 154.198.86.255


In [ ]:
import socket

def punch_hole_and_connect(my_id, server_ip, server_port):
    # Create a UDP socket and bind to any available port.
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.bind(("", 0))
    my_port = sock.getsockname()[1]
    print(f"My UDP port: {my_port}")

    # Register with the rendezvous server.
    register_message = f"REGISTER {my_id}"
    sock.sendto(register_message.encode(), (server_ip, server_port))
    
    # Wait for a response from the server.
    data, addr = sock.recvfrom(1024)
    print("Server response:", data.decode())
    
    # Wait for peer info from the server.
    data, addr = sock.recvfrom(1024)
    msg = data.decode().strip()
    tokens = msg.split()
    if tokens[0].upper() == "PEER" and len(tokens) >= 3:
        peer_ip = tokens[1]
        peer_port = int(tokens[2])
        print(f"Peer info: {peer_ip}:{peer_port}")
        
        # Send a UDP packet to the peer to punch a hole in the NAT.
        sock.sendto(b"Hello from " + my_id.encode(), (peer_ip, peer_port))
        print(f"Sent punch packet to {peer_ip}:{peer_port}")
        
        # Optionally, wait for a response from the peer.
        data, addr = sock.recvfrom(1024)
        print("Received reply from peer:", data.decode())
    else:
        print("Did not receive valid peer info.")

if __name__ == '__main__':
    my_id = input("Enter your client id (e.g., client1 or client2): ").strip()
    server_ip = '154.198.86.255' # input("Enter the rendezvous server IP: ").strip()
    server_port = 12345 # int(input("Enter the rendezvous server UDP port: ").strip())
    punch_hole_and_connect(my_id, server_ip, server_port)


My UDP port: 51724


In [3]:
import socket

def punch_hole_and_send(remote_ip, remote_port, message="Hello from Computer A"):
    # Create a UDP socket and bind it to any available local port.
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.bind(("", 0))  # Let the OS assign an available port
    local_port = sock.getsockname()[1]
    print(f"My local UDP port is {local_port}")

    # Send a UDP packet to Computer B's public IP and known port.
    sock.sendto(message.encode(), (remote_ip, remote_port))
    print(f"Sent message to {remote_ip}:{remote_port}")

    # Wait for a reply from Computer B.
    try:
        sock.settimeout(5)  # timeout after 5 seconds
        data, addr = sock.recvfrom(1024)
        print("Received reply from", addr, ":", data.decode())
    except socket.timeout:
        print("No response received.")
    finally:
        sock.close()

if __name__ == "__main__":
    remote_ip = "154.198.86.255" # input("Enter the remote computer's public IP: ").strip()
    remote_port = 5005 # int(input("Enter the remote computer's UDP port (e.g., 5005): ").strip())
    punch_hole_and_send(remote_ip, remote_port)


My local UDP port is 51729
Sent message to 154.198.86.255:5005
No response received.


In [11]:
import socket
import sys
import threading

rendezvous = ('103.121.178.245', 55555)

# connect to rendezvous
print('connecting to rendezvous server')

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind(('0.0.0.0', 50001))
sock.sendto(b'0', rendezvous)


connecting to rendezvous server


OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted

In [ ]:

while True:
    data = sock.recv(1024).decode()

    if data.strip() == 'ready':
        print('checked in with server, waiting')
        break

data = sock.recv(1024).decode()
ip, sport, dport = data.split(' ')
sport = int(sport)
dport = int(dport)

print('\ngot peer')
print('  ip:          {}'.format(ip))
print('  source port: {}'.format(sport))
print('  dest port:   {}\n'.format(dport))

# punch hole
# equiv: echo 'punch hole' | nc -u -p 50001 x.x.x.x 50002
print('punching hole')

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind(('0.0.0.0', sport))
sock.sendto(b'0', (ip, dport))

print('ready to exchange messages\n')

# listen for
# equiv: nc -u -l 50001
def listen():
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.bind(('0.0.0.0', sport))

    while True:
        data = sock.recv(1024)
        print('\rpeer: {}\n> '.format(data.decode()), end='')

listener = threading.Thread(target=listen, daemon=True);
listener.start()

# send messages
# equiv: echo 'xxx' | nc -u -p 50002 x.x.x.x 50001
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind(('0.0.0.0', dport))

while True:
    msg = input('> ')
    sock.sendto(msg.encode(), (ip, sport))

In [2]:
!pip install twisted[tls]

  Using cached twisted-24.11.0-py3-none-any.whl.metadata (20 kB)
  Using cached pyOpenSSL-25.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached service_identity-24.2.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached cryptography-44.0.1-cp39-abi3-win_amd64.whl.metadata (5.7 kB)
Using cached pyOpenSSL-25.0.0-py3-none-any.whl (56 kB)
Using cached service_identity-24.2.0-py3-none-any.whl (11 kB)
Using cached twisted-24.11.0-py3-none-any.whl (3.2 MB)
Using cached cryptography-44.0.1-cp39-abi3-win_amd64.whl (3.2 MB)


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python311\\Scripts\\cftp.exe' -> 'C:\\Python311\\Scripts\\cftp.exe.deleteme'


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from  twisted.internet.protocol import DatagramProtocol


ModuleNotFoundError: No module named 'twisted'